In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./1.make.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_make = df['Make'].tolist()

print('总数量：' + str(len(list_make)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for make in list_make:
    work.put_nowait(make)

def crawler():
    global df
    
    while not work.empty():
        make = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                data = 'func=navnode_fetch&payload=' + quote('{"jsn":{"tab":"catalog","make":"') + make.replace(' ', '+') + quote('","nodetype":"make"}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'},
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_year = html.xpath('//td[@class="nlabel"]/a/text()')
                
                # = = = = = = = = = = = = = = =
                
                list_url = html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')

                list_url = ['https://www.rockauto.com'+_ for _ in list_url]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Make': make,
                                        'Year': list_year,
                                        'Url': list_url})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Make': make}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(make +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./year_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')